In [1]:
# IMPORT LIBRARIES
import psycopg2 as pg2
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import snowflake.connector

In [2]:
#CONNECT TO SNOWFLAKE WITH REPORTER CREDENTIALS
conn = snowflake.connector.connect(
    user='emirkabasoglu', #emirkabasoglu
    password='7Bb13287', #7Bb13287
    account='se63477.north-europe.azure',
    warehouse='TRANSFORMING_WH',
    database='PROD_ANALYTICS_DB'
    )

In [3]:
#START THE CURSOR AND RUN THE QUERY
cur = conn.cursor()
cur.execute('''--CUSTOMERS MAP
            
SELECT
T1.CUSTOMER_NAME,
T1.BILLING_ADDRESS_COUNTRY,
T1.REGION,
T2.billing_address_city,
T2.billing_address_state,
T3.SUB_INDUSTRY,
T3.FTE_SIZE,
T6.ACCOUNT_STATUS,
t7.country_name,
ZEROIFNULL(T5.CURRENT_MONTH_CONTRACT_VALUE) AS CURRENT_MONTH_CONTRACT_VALUE

FROM PROD_ANALYTICS_DB.DBT_DEPLOY.CUSTOMERS T1
LEFT JOIN RAW_DB.CUSTOM_CHARGEBEE.customer T2 ON T1.CUSTOMER_ID = T2.id
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.INT_CUSTOMER_FIRMOGRAPHIC_ASSIGNED T3 ON T3.COMPANY_ID =T1.HUBSPOT_COMPANY_ID
LEFT JOIN COMMON_DB.STATIC.GICS T4 ON T4.SUB_INDUSTRY = T3.SUB_INDUSTRY
LEFT JOIN ( SELECT T2.CUSTOMER_ID, T1.DATE, SUM(T1.ARR_EUR) AS CURRENT_MONTH_CONTRACT_VALUE
            FROM PROD_ANALYTICS_DB.DBT_DEPLOY.ARR_SUBSCRIPTION_VALUES T1
            LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.SUBSCRIPTIONS T2 ON T1.SUBSCRIPTION_ID = T2.SUBSCRIPTION_ID
            WHERE T1.DATE = LAST_DAY(CURRENT_DATE()) AND T2.status != 'cancelled'
            GROUP BY 1,2    ) T5 ON T5.CUSTOMER_ID = T1.CUSTOMER_ID
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.INT_CUSTOMER_STATUS_CALCULATED T6 ON T6.CUSTOMER_ID = T1.CUSTOMER_ID
LEFT JOIN DBT_DEPLOY.countries_and_regions t7 on t1.billing_address_country =t7.country_id 
WHERE T6.ACCOUNT_STATUS !='cancelled'


''')


In [4]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)

In [5]:
# ASSIGN TO MAIN DF FOR FUTHER TRANSFORMATION
df_main = df_items_db_fetched

In [6]:
df_main.columns

Index(['CUSTOMER_NAME', 'BILLING_ADDRESS_COUNTRY', 'REGION',
       'BILLING_ADDRESS_CITY', 'BILLING_ADDRESS_STATE', 'SUB_INDUSTRY',
       'FTE_SIZE', 'ACCOUNT_STATUS', 'COUNTRY_NAME',
       'CURRENT_MONTH_CONTRACT_VALUE'],
      dtype='object')

In [7]:
#CONVERT ALL THE COLUMN NAMES LOWER CASE

df_main.columns= df_main.columns.str.lower()


In [8]:
#Formats an integer with commas as thousand separators.

df_main.style.format({"CURRENT_MONTH_CONTRACT_VALUE": "{:,.0f}"})

,customer_name,billing_address_country,region,billing_address_city,billing_address_state,sub_industry,fte_size,account_status,country_name,current_month_contract_value
0,Nuuday A/S,DK,NORBEN,Kobenhavn SV,None,COMPUTER_SOFTWARE,1000,active,Denmark,36448.451600
1,African American Chamber of Commerce of New Jersey,US,US,Trenton,New Jersey,NON_PROFIT_ORGANIZATION_MANAGEMENT,10,active,United States,19432.526368
2,JanCo FS3,US,US,Atlanta,Georgia,LEGAL_SERVICES,1,active,United States,48698.441680
3,Moerlein Lager House,US,US,Cincinatti,Ohio,RESTAURANTS,100,active,United States,7292.736000
4,Amway GmbH,DE,DACH,Puchheim,None,RETAIL,15000,active,Germany,7364.558400
5,BarMa - GRAM Group,DK,NORBEN,Brabrand,None,HOSPITALITY,50,non_renewing,Denmark,12192.070000
6,Hotel Keflavik ehf,IS,NORBEN,Reykjanesbær,None,HOSPITALITY,50,active,Iceland,8220.000000
7,Destiny Student,GB,UK,Edinburgh,None,HOSPITALITY,50,active,United Kingdom,7385.568900
8,Davidsens Tømmerhandel A/S,DK,NORBEN,Vojens,None,BUILDING_MATERIALS,1000,active,Denmark,21055.980000
9,Landsvirkjun,IS,NORBEN,Reykjavik,None,RENEWABLES_ENVIRONMENT,500,active,Iceland,19682.493120


In [9]:
df_main.head(2)

,customer_name,billing_address_country,region,billing_address_city,billing_address_state,sub_industry,fte_size,account_status,country_name,current_month_contract_value
0,Nuuday A/S,DK,NORBEN,Kobenhavn SV,None,COMPUTER_SOFTWARE,1000,active,Denmark,36448.451600
1,African American Chamber of Commerce of New Je...,US,US,Trenton,New Jersey,NON_PROFIT_ORGANIZATION_MANAGEMENT,10,active,United States,19432.526368


In [10]:
# SET UP DATE FOR NAMING THE OUTPUT FILES
from dateutil.relativedelta import relativedelta
period_end_date = date.today() - relativedelta(months=1)
report_date = period_end_date.strftime('%b-%y')
report_date

'Aug-24'

In [11]:
excelfilename = report_date +"-Customers Map.csv"

In [12]:
# OUTPUT FINAL FILE WITH ALL DIFFERENCES
df_main.to_csv(excelfilename,  index=False)
